In [28]:
import os
import sys
sys.path.append('../')

import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from pprint import pprint
import scipy

import src.io as sio
import src.preprocessing as spp
import src.fitting as sft

In [2]:
AFM_FOLDER = "20200818_Akiyama_AFM/"
AFM_FOLDER1 = "20200721_Akiyama_AFM/"
AFM_FOLDER2 = "20200824_Akiyama_AFM/"

## 20200721_Akiyama_AFM

In [ ]:
params, data = sio.read_dat(AFM_FOLDER1 + "frq-sweep002.dat")
freq_shift = data["Frequency Shift (Hz)"].values
amplitude = data["Amplitude (m)"].values
phase = data["Phase (deg)"].values
amp_freq_sweep = sft.fit_lorentzian(freq_shift, amplitude, linear_offset=True)
phase_freq_sweep = sft.fit_fano(freq_shift, phase)

In [ ]:
%matplotlib inline

fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.plot(freq_shift, amplitude)
ax1.plot(freq_shift, amp_freq_sweep.best_fit)
ax1.set_ylabel(data.columns[2])

ax2.plot(freq_shift, phase)
ax2.plot(freq_shift, phase_freq_sweep.best_fit)
ax2.set_ylabel(data.columns[3])
ax2.set_xlabel(data.columns[0])

Quality factor can be calculated as $ Q = \frac{f_R}{\Delta f} $

In [ ]:
print(f'Q-factor= {params["f_res (Hz)"] / amp_freq_sweep.params["fwhm"].value}')

## 20200818_Akiyama_AFM

In [ ]:
params, data = sio.read_dat(AFM_FOLDER + "frq-sweep001.dat")
#pprint(params, sort_dicts=False)
freq_shift = data["Frequency Shift (Hz)"]
amplitude = data["Amplitude (m)"]
phase = data["Phase (deg)"]
fano = sft.fit_fano(freq_shift, amplitude)
lorentzian = sft.fit_fano(freq_shift, phase)
params

## Equations for calculating Q factor

$$ Q = \frac{f_R}{\Delta f} $$

$$ Q = \frac{A(\omega_0)}{A_{in}} $$

In [ ]:
f_res = 44379.7064
sigma = 62.2841355
print(f_res/sigma)

A_drive = 50e-3
A_res = 28.3e-6 * 1 / 500e-6
print(A_res/A_drive)

# Calibration
A_drive = 50e-3
osc_amp = 50e-9

print(osc_amp/A_drive)

## Plot frequency sweep curves

In [ ]:
%matplotlib widget
fig, (ax1, ax2) = plt.subplots(nrows=2, sharex=True)
ax1.plot(freq_shift, amplitude)
ax1.plot(freq_shift, fano.best_fit)
ax1.set_ylabel(data.columns[2])

ax2.plot(freq_shift, phase)
ax2.plot(freq_shift, lorentzian.best_fit)
ax2.set_ylabel(data.columns[3])
ax2.set_xlabel(data.columns[1])

## Extract fit values

In [ ]:
print("{} = {:.1f} +- {:.1f}".format(fano.params["sigma"].name, fano.params["sigma"].value, fano.params["sigma"].stderr))
print("{} = {:.2e} +- {:.0e}".format(fano.params["amplitude"].name, fano.params["amplitude"].value, fano.params["amplitude"].stderr))

# 20200824_Akiyama_AFM

## Loop to automatically read files from disk

Reads all files stored in **AFM_FOLDER2 = "20200824_Akiyama_AFM/"** and plots the amplitude and phase data.

Optionally, the data can be fit to Fano resonances by setting the variable
```python
fit = True
```

### Q-factor
$$ Q = \frac{f_R}{\Delta f} = \frac{f_R}{2 \sigma} $$

Errors are calculated as (this also gives an estimate of the SNR):
$$ \frac{\Delta Q}{Q} = \sqrt{ \left( \frac{\Delta (\Delta f)}{\Delta f} \right)^2 + \left( \frac{\Delta (\sigma)}{\sigma} \right)^2 } $$

### SNR
best fit values / standard error of the residuals

These are better viewed as relative values than as absolute measures.

### Chi-square
Another estimate of the SNR, is the reduced Chi square (lower is better):
$$ \chi^2_\nu = \frac{\chi^2} \nu $$
where chi-squared is a weighted sum of squared deviations:
$$ \chi^2 = \sum_{i} {\frac{(O_i - C_i)^2}{\sigma_i^2}} $$

In [91]:
%matplotlib widget

fit = True    # Setting to True will take slightly longer due to the fitting protocols

files = []
for file in os.listdir("../../Data/" + AFM_FOLDER2):
    if file.endswith(".dat"):
        files.append(file)

fig, ax = plt.subplots(nrows=len(files), ncols=2)

for idx, file in enumerate(files):
    params, data = sio.read_dat(AFM_FOLDER2 + file)
    freq_shift = data["Frequency Shift (Hz)"]
    amplitude = data["Amplitude (m)"]
    phase = data["Phase (deg)"]

    ax[idx, 0].plot(freq_shift, amplitude)
    ax[idx, 0].set_ylabel(data.columns[2])
    ax[idx, 0].set_title(file)

    ax[idx, 1].plot(freq_shift, phase)
    ax[idx, 1].set_ylabel(data.columns[3])
    ax[idx, 1].set_title(file)
    
    if fit:
        fano1 = sft.fit_fano(freq_shift, amplitude)
        snr = np.mean(scipy.stats.sem(fano1.best_fit/fano1.residual, axis=None, ddof=0))
        q_factor = (params["Center Frequency (Hz)"] + fano1.params["center"].value) / (2 * fano1.params["sigma"].value)
        q_factor_err = q_factor * np.sqrt((fano1.params["center"].stderr/fano1.params["center"].value)**2 + (fano1.params["sigma"].stderr/fano1.params["sigma"].value)**2)
        ax[idx, 0].plot(freq_shift, fano1.best_fit, label="Q={:.0f}$\pm{:.0f}$".format(q_factor, q_factor_err))
        ax[idx, 0].legend()
        fano2 = sft.fit_fano(freq_shift, phase, linear_offset=True)
        ax[idx, 1].plot(freq_shift, fano2.best_fit)
        print("({}) SNR = {:.0f}, reduced-chi-square = {:.2e}".format(file, snr, fano1.redchi))
        fig.tight_layout()

fig.text(0.5, 0.02, data.columns[1], ha='center', va='center')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

(frq-sweep001.dat) SNR = 1948, reduced-chi-square = 1.26e-22
(frq-sweep002.dat) SNR = 2571, reduced-chi-square = 1.51e-22
(frq-sweep003.dat) SNR = 4116, reduced-chi-square = 2.88e-23


Text(0.5, 0.02, 'Center Frequency (Hz)')

In [88]:
fano1